In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime
import warnings
from colorama import Fore, Style
import gc
import pickle

from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibrationDisplay
import lightgbm
from catboost import Pool, CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
import catboost
from utils import get_data

In [2]:
test_data = get_data('data/test_data.ftr')

with open('models/feature_list.pkl', 'rb') as f:
    feature_list, cat_features = pickle.load(f)

not_found = []
for feature in feature_list:
    if feature not in test_data.columns.tolist():
        not_found.append(feature)
test_data = test_data.assign(**dict.fromkeys(not_found, 0))

y_pred_list = []

for i in range(5):
    model = CatBoostClassifier()
    model.load_model(f'models/catboost_{i}.cbm')

    y_pred_list.append(model.predict_proba(Pool(test_data[feature_list], cat_features=cat_features))[:,1])

sub = pd.DataFrame({'customer_ID': test_data.index,
                    'prediction': np.mean(y_pred_list, axis=0)})
sub.to_csv('submission_v6.csv', index=False)
display(sub)

/Users/dmitrykutsenko/Desktop/My_python/Kaggle_amex/utils.py:79: FutureWarning: ['S_2', 'D_63', 'D_64', 'D_66', 'D_68', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  main_data = data.groupby('customer_ID').agg(['mean', 'median', 'std', 'min', 'max']).sort_index()


,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.025810
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.001577
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.034449
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.205410
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.838726
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.011544
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.767213
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.375795
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.273817
